In [1]:
import pyglet
import math
import time
import serial
import matplotlib.pyplot as plt
import numpy as np
import time
#from datetime import datetime

# run following code on the terminal if 'pyglet' is not installed
# conda install -c conda-forge pyglet

# Load the white noise audio file
player = pyglet.media.Player()
source = pyglet.media.load('./new_whitenoise.wav', streaming=False) 
player.queue(source)


# Load the audio file for warning
fast_player = pyglet.media.Player()
fast_source = pyglet.media.load('./fast.wav')
fast_player.queue(fast_source)




In [2]:
#change the audio feedback pitch based on breathing data
#give audio feedback when breathing frequency is too high

warning_timer_start=time.time()
connected_flag = False


#establish connect with arduino
while not connected_flag:
    time.sleep(2)
    try:
        ser = serial.Serial('COM3') # open serial port
        if (not ser)==False:
            connected_flag=True;            
            print(ser.name) 
    except Exception:
        pass


count=0
BUF_SIZE=60 # 60 seconds window
SOUNDPLAY_PERIOD=1
filtered_breathing_data=[]
breathing_data=[]
buf_count=0
SAMPLING_FREQ=20 #sample at 20HZ
flag=False
breath_count=0

half_period=0
start_flag=0
end_flag=0
peak_flag=0
valley_flag=0
breath_freq=8
f=open('output.txt','w+')

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

    
    
while True: 
    try:
        time.sleep(1/SAMPLING_FREQ)
        if count>=SOUNDPLAY_PERIOD: # for adjusting the sound volume
            count=0

            if len(breathing_data)==BUF_SIZE:
                temp=moving_average(breathing_data, n=20)
                if max(temp)-min(temp) !=0:
                    coef=1-0.95*((temp[-1]-min(temp))/(max(temp)-min(temp)))
                else:
                    coef=1
                    
                player.volume=1
                player.pitch = coef
                player.play() 
                print("temp:",temp[-1])


                # check if the wave reaches its peak or valley
                if 0.99<=coef and coef<=1.01: #reach peak range
                    peak_flag=1
                elif 0.049<=coef and coef<=0.051: #reach valley range
                    valley_flag=1

                    
                #if peak is reached, start calculating its period. timer starts
                if peak_flag==1 and start_flag==0:
                    start=time.time()
                    start_flag=1

                #half period reached
                if peak_flag==1 and valley_flag==1:
                    half_period=1
                    peak_flag=0
                    valley_flag=0
                    
                #one period reached, stop timer
                if half_period==1 and peak_flag==1 and start_flag==1:
                    end=time.time()
                    breath_freq=end-start
                    start_flag=0
                    half_period=0
                    print("breath_freq: ",breath_freq)

                #only check every 1 minutes
                warning_timer_end=time.time()
                print('warning time acc: ',warning_timer_end-warning_timer_start)
                if (warning_timer_end-warning_timer_start)>=60:   
                    evaluate=True
                    i=0
                    while evaluate==True:
                        if breath_freq<8:

                            print("breath too fast")
                            fast_player.volume=1
                            fast_player.play()
                            current_time=time.time()
                            if i==0:
                                fast_play_start=time.time()

                            if current_time-fast_play_start>=3.9:

                                evaluate=False
                                fast_player.pause()
                                break


                        else:
                            evaluate=False

                        i+=1


                    #restart timer
                    warning_timer_start=time.time()

            print("preak_flag:",peak_flag,"valley_flag:",valley_flag,"half_period:",half_period,"start_flag:",start_flag,"breath_freq:",breath_freq)

  

        if buf_count>=BUF_SIZE: 
            del breathing_data[0]

        else:
            buf_count+=1      

        x=ser.readline().decode('utf-8') 
        ts = time.time_ns()
        print (x)
        f.write(str(ts) + " " + x)

        breathing_data.append(int(x))

        count+=1
    
    except KeyboardInterrupt:

        f.close()
        player.exit()
        pass

COM3
541

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
541

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
538

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
538

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
535

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
535

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
536

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
534

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
537

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
539

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
542

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
544

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 8
545

preak_flag: 0 valley_flag: 0 half_period: 0 start_flag

536

temp: 541.0
warning time acc:  11.414990663528442
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
534

temp: 540.65
warning time acc:  11.509773254394531
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
535

temp: 540.3
warning time acc:  11.610705852508545
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
532

temp: 539.6
warning time acc:  11.705910444259644
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
531

temp: 538.85
warning time acc:  11.809376955032349
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
531

temp: 538.4
warning time acc:  11.910748958587646
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
540

temp: 538.2
warning time acc:  12.007230043411255
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
538

temp: 538.05
warning time acc:  12.107873678207397
preak_flag: 0 valley_flag: 1 half_period: 1 st

temp: 657.15
warning time acc:  17.826560735702515
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
633

temp: 654.55
warning time acc:  17.92315673828125
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
631

temp: 652.3
warning time acc:  18.02152180671692
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
634

temp: 649.95
warning time acc:  18.12418031692505
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 8
630

temp: 647.95
warning time acc:  18.22404408454895
preak_flag: 0 valley_flag: 0 half_period: 1 start_flag: 1 breath_freq: 8
627

temp: 646.3
breath_freq:  9.217246055603027
warning time acc:  18.326171159744263
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 0 breath_freq: 9.217246055603027
625

temp: 644.75
warning time acc:  18.424509286880493
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 9.217246055603027
624

temp: 642.9
warning time acc:  18.52488

592

temp: 598.95
warning time acc:  23.638179302215576
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 9.217246055603027
592

temp: 598.45
warning time acc:  23.738171100616455
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 9.217246055603027
595

temp: 598.0
warning time acc:  23.83701729774475
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 9.217246055603027
606

temp: 598.1
warning time acc:  23.93880319595337
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 9.217246055603027
601

temp: 597.95
warning time acc:  24.040208101272583
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 9.217246055603027
601

temp: 597.75
warning time acc:  24.13738775253296
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 9.217246055603027
598

temp: 597.55
warning time acc:  24.237733602523804
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 9.217246

temp: 637.1
warning time acc:  29.45247507095337
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 9.217246055603027
679

temp: 641.65
warning time acc:  29.55046010017395
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 9.217246055603027
722

temp: 648.3
warning time acc:  29.650825023651123
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 9.217246055603027
752

temp: 656.25
warning time acc:  29.75300168991089
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 9.217246055603027
751

temp: 664.3
warning time acc:  29.85168194770813
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 9.217246055603027
731

temp: 671.4
warning time acc:  29.954660177230835
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 9.217246055603027
694

temp: 676.65
warning time acc:  30.053261041641235
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 9.21724605560302

...

742

temp: 719.2
warning time acc:  21.650573015213013
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 4.9152655601501465
702

temp: 718.7
warning time acc:  21.746384859085083
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 4.9152655601501465
676

temp: 717.6
warning time acc:  21.853750467300415
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 4.9152655601501465
688

temp: 716.9
warning time acc:  21.946796417236328
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 4.9152655601501465
732

temp: 718.15
warning time acc:  22.05097532272339
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 4.9152655601501465
730

temp: 721.1
warning time acc:  22.14648151397705
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 4.9152655601501465
765

temp: 724.85
warning time acc:  22.260382890701294
preak_flag: 0 valley_flag: 1 half_period: 1 start_flag: 1 breath_freq: 4.91

temp: 595.25
warning time acc:  27.657262325286865
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.0899131298065186
579

temp: 594.45
warning time acc:  27.74526858329773
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.0899131298065186
577

temp: 593.5
warning time acc:  27.839261054992676
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.0899131298065186
573

temp: 592.45
warning time acc:  27.92227816581726
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.0899131298065186
591

temp: 592.35
warning time acc:  28.004271030426025
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.0899131298065186
590

temp: 591.95
warning time acc:  28.0992751121521
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.0899131298065186
585

temp: 591.15
warning time acc:  28.184280395507812
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.08991

temp: 574.2
warning time acc:  33.28758215904236
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
568

temp: 573.4
warning time acc:  33.38058400154114
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
567

temp: 572.6
warning time acc:  33.475594997406006
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
567

temp: 572.0
warning time acc:  33.576597452163696
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
566

temp: 571.35
warning time acc:  33.68860459327698
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
566

temp: 570.75
warning time acc:  33.776610374450684
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
565

temp: 570.15
warning time acc:  33.877622842788696
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.7287118434906

554

temp: 554.5
warning time acc:  38.98916411399841
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
553

temp: 554.35
warning time acc:  39.09016728401184
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
552

temp: 554.15
warning time acc:  39.19017314910889
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
552

temp: 554.1
warning time acc:  39.30218553543091
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
551

temp: 553.85
warning time acc:  39.399186849594116
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
552

temp: 553.65
warning time acc:  39.5001916885376
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.728711843490601
552

temp: 553.45
warning time acc:  39.59819769859314
preak_flag: 1 valley_flag: 0 half_period: 0 start_flag: 1 breath_freq: 7.7287118434

KeyboardInterrupt: 